In [3]:
%matplotlib inline
import qiime2
from qiime2 import Artifact
from biom import load_table
from tempfile import mkdtemp
from qiime2.plugins import demux, deblur, quality_filter, \
                           metadata, feature_table, alignment, \
                           phylogeny, diversity, emperor, feature_classifier, \
                           taxa, composition, woltka, empress

from biom import load_table
from deicode.rpca import auto_rpca
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from skbio.stats.distance import permanova
from skbio.stats.distance import permdisp

In [2]:
from os.path import abspath, join

In [4]:
from qiime2 import Visualization

In [12]:
from skbio import TreeNode
import skbio as skb

In [5]:
bacteria='Akkermansia_muciniphila' ## interested bacteria
key=True

In [6]:
from os.path import abspath, join
bacteria_trimmed=bacteria[0]+bacteria[bacteria.find("_"):len(bacteria)]
indir = abspath('/panfs/panfs1.ucsd.edu/panscratch/jhc103/VertMetaphlan-frmerged/output_'+bacteria_trimmed+"/") ## input directory
outdir='/panfs/panfs1.ucsd.edu/panscratch/jhc103/VertMetaphlan-frmerged/empress-output' ## Path to output directory
bacteria_tree_fp = join(indir, 'RAxML_bestTree.s__'+bacteria+'.StrainPhlAn3.tre') ## Path to strainphlan tree

In [13]:
sample_tree= skb.io.read(bacteria_tree_fp, format='newick', 
               into=TreeNode,
               convert_underscores=False)

In [7]:
md_dir = '/panfs/panfs1.ucsd.edu/panscratch/jhc103/VertMetaphlan-frmerged/metadata'
host_md_fp = join(md_dir, 'vert_metadata_new_empress_strain_tree.txt') ## path to metadata

### Import host tree

In [31]:
tree_dir = abspath('/panfs/panfs1.ucsd.edu/panscratch/jhc103/VertMetaphlan-frmerged')
host_tree_fp = join(tree_dir, 'host_tree_list.nwk')
host_tree_orig= skb.io.read(host_tree_fp, format='newick', 
                       into=TreeNode,
                       convert_underscores=False)

# Subsample tree to 1 sample per species
### Import metadata

In [8]:
md_dir = '/panfs/panfs1.ucsd.edu/panscratch/jhc103/VertMetaphlan-frmerged/metadata'
host_md_fp = join(md_dir, 'vert_metadata_8_23.txt')
host_md_orig = pd.read_csv(host_md_fp, sep='\t', encoding='windows-1252')

### Import & sample tree

In [14]:
sample_tips = [x.name for x in sample_tree.tips()]

In [15]:
## Shear sample tree
host_md=host_md_orig
host_md_subset =  host_md.loc[(host_md['SampleID2'].isin(sample_tips)),].groupby('TimeTree_returned').sample(1).reset_index()

sheared_sample_tree = sample_tree.shear(host_md_subset['SampleID2'])

In [18]:
host_md_subset =  host_md.loc[(host_md['SampleID2'].isin(sample_tips)),].groupby('TimeTree_returned').sample(1).reset_index()
sheared_sample_tree = sample_tree.shear(host_md_subset['SampleID2'])

'                              /-Homo-sapiens_SAMEA6809425.ebi_filtered_r1r2combined_sampleCombined\n                             |\n                             |                    /-Choloepus-didactylus_SAMEA6809544.ebi_filtered_r1r2combined_sampleCombined\n                    /--------|          /--------|\n                   |         |         |          \\-Choloepus-hoffmanni_SAMEA6809527.ebi_filtered_r1r2combined_sampleCombined\n                   |         |         |\n                   |          \\--------|                    /-Trichechus-manatus_Song75_ace_host_filtered_r1r2combined\n                   |                   |          /--------|\n                   |                   |         |          \\-Eudromia-elegans_S_S_Temp_D706-AK1544_filtered_r1r2combined\n                   |                    \\--------|\n          /--------|                             |          /-Strix-uralensis_SAMEA6809608.ebi_filtered_r1r2combined_sampleCombined\n         |         |    

In [20]:
bacteria_subtree_fp = join(indir, bacteria+'_subsampled_sample_tree.tre') ## Path to strainphlan tree

In [21]:
skb.io.write(sheared_sample_tree, "newick", bacteria_subtree_fp)

'/panfs/panfs1.ucsd.edu/panscratch/jhc103/VertMetaphlan-frmerged/output_A_muciniphila/Akkermansia_muciniphila_subsampled_sample_tree.tre'

In [92]:
host_md_subset

,index,sample_name,studyID,SampleID3,SampleID2,file_filtered_r1r2_combined,reads_per_unfiltered_file,reads_per_file,species_id,host_phylum,...,host_diet,country,habitat,sample_type,diet_category_12,diet_category_3,captive_wild,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,409,13881.SAMEA6809531,13881.0,Bradypus-variegatus_SAMEA6809531.ebi_filtered_...,Bradypus-variegatus_SAMEA6809531.ebi_filtered_...,Bradypus-variegatus_SAMEA6809531.ebi_filtered_...,4687274.0,4681548.0,brown-throated_sloth.1,Chordata,...,herbivore,Nicaragua,NaN,feces,herbivore,herbivore,wild,NaN,NaN,NaN
1,536,13881.SAMEA6809488,13881.0,Chalcites-basalis_SAMEA6809488.ebi_filtered_r1...,Chalcites-basalis_SAMEA6809488.ebi_filtered_r1...,Chalcites-basalis_SAMEA6809488.ebi_filtered_r1...,2010544.0,2010106.0,horsfields_bronze_cuckoo.1,Chordata,...,carnivore,Australia,NaN,feces,insectivore,carnivore,wild,NaN,NaN,NaN
2,621,13881.SAMEA6809544,13881.0,Choloepus-didactylus_SAMEA6809544.ebi_filtered...,Choloepus-didactylus_SAMEA6809544.ebi_filtered...,Choloepus-didactylus_SAMEA6809544.ebi_filtered...,11405774.0,11086930.0,southern_two-toed_sloth.1,Chordata,...,herbivore,Austria,NaN,feces,folivore,herbivore,captive,NaN,NaN,NaN
3,532,13881.SAMEA6809527,13881.0,Choloepus-hoffmanni_SAMEA6809527.ebi_filtered_...,Choloepus-hoffmanni_SAMEA6809527.ebi_filtered_...,Choloepus-hoffmanni_SAMEA6809527.ebi_filtered_...,1983646.0,1983064.0,hoffmann's_two-fingered_sloth.1,Chordata,...,herbivore,Nicaragua,NaN,feces,herbivore,herbivore,wild,NaN,NaN,NaN
4,84,11166.NATL.25082.tinamou,11166.0,Eudromia-elegans_S_S_Temp_D706-AK1544_filtered...,Eudromia-elegans_S_S_Temp_D706-AK1544_filtered...,Eudromia-elegans_S_S_Temp_D706-AK1544_filtered...,52067270.0,51754556.0,tinamou.1,Chordata,...,NaN,USA,NaN,NaN,omnivore,omnivore,captive,NaN,NaN,NaN
5,644,13881.SAMEA6809515,13881.0,Gorilla-gorilla_SAMEA6809515.ebi_filtered_r1r2...,Gorilla-gorilla_SAMEA6809515.ebi_filtered_r1r2...,Gorilla-gorilla_SAMEA6809515.ebi_filtered_r1r2...,4459468.0,4411764.0,western_lowland_gorilla.2,Chordata,...,herbivore,Republic of the Congo,NaN,feces,herbivore,herbivore,wild,NaN,NaN,NaN
6,543,13881.SAMEA6809425,13881.0,Homo-sapiens_SAMEA6809425.ebi_filtered_r1r2int...,Homo-sapiens_SAMEA6809425.ebi_filtered_r1r2com...,Homo-sapiens_SAMEA6809425.ebi_filtered_r1r2com...,55200616.0,55172154.0,human.3,Chordata,...,omnivore,Austria,NaN,feces,omnivore,omnivore,human,NaN,NaN,NaN
7,322,13114.song.51.s004,13114.0,Lemur-catta_Song51_24592_host_filtered_r1r2int...,Lemur-catta_Song51_24592_host_filtered_r1r2com...,Lemur-catta_Song51_24592_host_filtered_r1r2com...,18513374.0,18510390.0,ringtailed_lemur.1,Chordata,...,herbivore,USA,NaN,NaN,omnivore,omnivore,captive,NaN,NaN,NaN
8,77,11166.AR.M18.flyingsquirrel,11166.0,Pteromys-volans_S_S_Temp_D707-AK1545_filtered_...,Pteromys-volans_S_S_Temp_D707-AK1545_filtered_...,Pteromys-volans_S_S_Temp_D707-AK1545_filtered_...,20695232.0,20495598.0,siberian_flying_squirrel.1,Chordata,...,NaN,Finland,NaN,NaN,herbivore,herbivore,wild,NaN,NaN,NaN
9,635,13881.SAMEA6809608,13881.0,Strix-uralensis_SAMEA6809608.ebi_filtered_r1r2...,Strix-uralensis_SAMEA6809608.ebi_filtered_r1r2...,Strix-uralensis_SAMEA6809608.ebi_filtered_r1r2...,3764164.0,3757742.0,ural_owl.3,Chordata,...,carnivore,Austria,NaN,feces,carnivore,carnivore,wild,NaN,NaN,NaN


### Share host tree

In [35]:
sheared_host_tree = host_tree_orig.shear(host_md_subset['TimeTree_returned'])

In [36]:
host_subtree_fp = join(indir, bacteria+'_subsampled_host_tree.tre') ## Path to strainphlan tree
skb.io.write(sheared_host_tree, "newick", host_subtree_fp)

'/panfs/panfs1.ucsd.edu/panscratch/jhc103/VertMetaphlan-frmerged/output_A_muciniphila/Akkermansia_muciniphila_subsampled_host_tree.tre'

## Convert newick tree to qza object 


In [37]:
if key:
    bacteria_tree=Artifact.import_data("Phylogeny[Rooted]", bacteria_subtree_fp, view_type="NewickFormat")
    bacteria_tree.save(indir+'/'+bacteria+'_subsampled_sample_tree')
    host_tree=Artifact.import_data("Phylogeny[Rooted]", host_subtree_fp, view_type="NewickFormat")
    host_tree.save(indir+'/'+bacteria+'_subsampled_host_tree')

In [52]:
!ls $indir

Akkermansia_muciniphila_subsampled_host_tree.qza
Akkermansia_muciniphila_subsampled_host_tree.tre
Akkermansia_muciniphila_subsampled_sample_tree.qza
Akkermansia_muciniphila_subsampled_sample_tree.tre
Akkermansia_muciniphila_subsampled_tree.qza
Akkermansia_muciniphila_tree.qza
RAxML_bestTree.s__Akkermansia_muciniphila.StrainPhlAn3.tre
RAxML_info.s__Akkermansia_muciniphila.StrainPhlAn3.tre
RAxML_log.s__Akkermansia_muciniphila.StrainPhlAn3.tre
RAxML_parsimonyTree.s__Akkermansia_muciniphila.StrainPhlAn3.tre
RAxML_result.s__Akkermansia_muciniphila.StrainPhlAn3.tre
s__Akkermansia_muciniphila.info
s__Akkermansia_muciniphila.polymorphic
s__Akkermansia_muciniphila.StrainPhlAn3_concatenated.aln


# Run empress community-plot
## Visualize sample tree

In [109]:
host_md = host_md_subset.iloc[:,0:28]
host_md.drop('index',axis=1,inplace=True)

In [110]:
sample_md=host_md.rename(columns={'sample_name':'sample_name_2'})
sample_md=sample_md.rename(columns={'SampleID2':'SampleID'})
sample_md=sample_md.set_index("SampleID")
sample_md

,sample_name_2,studyID,SampleID3,file_filtered_r1r2_combined,reads_per_unfiltered_file,reads_per_file,species_id,host_phylum,host_class,host_family,...,host_common_name,host_flight,host_modality,host_diet,country,habitat,sample_type,diet_category_12,diet_category_3,captive_wild
SampleID,,,,,,,,,,,,,,,,,,,,,
Bradypus-variegatus_SAMEA6809531.ebi_filtered_r1r2combined_sampleCombined,13881.SAMEA6809531,13881.0,Bradypus-variegatus_SAMEA6809531.ebi_filtered_...,Bradypus-variegatus_SAMEA6809531.ebi_filtered_...,4687274.0,4681548.0,brown-throated_sloth.1,Chordata,Mammalia,Bradypodidae,...,Brown-throated sloth,NaN,NaN,herbivore,Nicaragua,NaN,feces,herbivore,herbivore,wild
Chalcites-basalis_SAMEA6809488.ebi_filtered_r1r2combined_sampleCombined,13881.SAMEA6809488,13881.0,Chalcites-basalis_SAMEA6809488.ebi_filtered_r1...,Chalcites-basalis_SAMEA6809488.ebi_filtered_r1...,2010544.0,2010106.0,horsfields_bronze_cuckoo.1,Chordata,Aves,Cuculidae,...,Horsfields_Bronze_Cuckoo,NaN,NaN,carnivore,Australia,NaN,feces,insectivore,carnivore,wild
Choloepus-didactylus_SAMEA6809544.ebi_filtered_r1r2combined_sampleCombined,13881.SAMEA6809544,13881.0,Choloepus-didactylus_SAMEA6809544.ebi_filtered...,Choloepus-didactylus_SAMEA6809544.ebi_filtered...,11405774.0,11086930.0,southern_two-toed_sloth.1,Chordata,Mammalia,Megalonychidae,...,southern two-toed sloth,NaN,NaN,herbivore,Austria,NaN,feces,folivore,herbivore,captive
Choloepus-hoffmanni_SAMEA6809527.ebi_filtered_r1r2combined_sampleCombined,13881.SAMEA6809527,13881.0,Choloepus-hoffmanni_SAMEA6809527.ebi_filtered_...,Choloepus-hoffmanni_SAMEA6809527.ebi_filtered_...,1983646.0,1983064.0,hoffmann's_two-fingered_sloth.1,Chordata,Mammalia,Megalonychidae,...,Hoffmann's two-fingered sloth,NaN,NaN,herbivore,Nicaragua,NaN,feces,herbivore,herbivore,wild
Eudromia-elegans_S_S_Temp_D706-AK1544_filtered_r1r2combined,11166.NATL.25082.tinamou,11166.0,Eudromia-elegans_S_S_Temp_D706-AK1544_filtered...,Eudromia-elegans_S_S_Temp_D706-AK1544_filtered...,52067270.0,51754556.0,tinamou.1,Chordata,Aves,Tinamidae,...,tinamou,flighted,NaN,NaN,USA,NaN,NaN,omnivore,omnivore,captive
Gorilla-gorilla_SAMEA6809515.ebi_filtered_r1r2combined_sampleCombined,13881.SAMEA6809515,13881.0,Gorilla-gorilla_SAMEA6809515.ebi_filtered_r1r2...,Gorilla-gorilla_SAMEA6809515.ebi_filtered_r1r2...,4459468.0,4411764.0,western_lowland_gorilla.2,Chordata,Mammalia,Hominidae,...,western lowland gorilla,NaN,NaN,herbivore,Republic of the Congo,NaN,feces,herbivore,herbivore,wild
Homo-sapiens_SAMEA6809425.ebi_filtered_r1r2combined_sampleCombined,13881.SAMEA6809425,13881.0,Homo-sapiens_SAMEA6809425.ebi_filtered_r1r2int...,Homo-sapiens_SAMEA6809425.ebi_filtered_r1r2com...,55200616.0,55172154.0,human.3,Chordata,Mammalia,Hominidae,...,human,NaN,NaN,omnivore,Austria,NaN,feces,omnivore,omnivore,human
Lemur-catta_Song51_24592_host_filtered_r1r2combined,13114.song.51.s004,13114.0,Lemur-catta_Song51_24592_host_filtered_r1r2int...,Lemur-catta_Song51_24592_host_filtered_r1r2com...,18513374.0,18510390.0,ringtailed_lemur.1,Chordata,Mammalia,Lemuridae,...,Ringtailed lemur,flightless,"terrestrial, flightless",herbivore,USA,NaN,NaN,omnivore,omnivore,captive
Pteromys-volans_S_S_Temp_D707-AK1545_filtered_r1r2combined,11166.AR.M18.flyingsquirrel,11166.0,Pteromys-volans_S_S_Temp_D707-AK1545_filtered_...,Pteromys-volans_S_S_Temp_D707-AK1545_filtered_...,20695232.0,20495598.0,siberian_flying_squirrel.1,Chordata,Mammalia,Sciuridae,...,Siberian flying squirrel,flightless,NaN,NaN,Finland,NaN,NaN,herbivore,herbivore,wild


In [111]:
sample_metadata_fp = join(md_dir, "metadata_filename_as_SampleID.txt") ## Path to strainphlan tree
sample_md.to_csv(sample_metadata_fp, sep='\t')

In [112]:
!qiime empress tree-plot \
    --i-tree $indir"/Akkermansia_muciniphila_subsampled_tree.qza"\
    --m-feature-metadata-file $sample_metadata_fp \
    --o-visualization $outdir/A_muciniphila_tree.qzv

Saved Visualization to: /panfs/panfs1.ucsd.edu/panscratch/jhc103/VertMetaphlan-frmerged/empress-output/A_muciniphila_tree.qzv


In [113]:
## View the qza inline in jupyternotebook 
viz = Visualization.load('/panfs/panfs1.ucsd.edu/panscratch/jhc103/VertMetaphlan-frmerged/empress-output/A_muciniphila_tree.qzv')
viz

<visualization: Visualization uuid: e8931332-bb3b-4136-838d-07372f8413a8>

## Visualize host tree

In [114]:
host_subset_md=host_md.rename(columns={'sample_name':'sample_name_2'})
host_subset_md=host_subset_md.rename(columns={'TimeTree_returned':'SampleID'})
host_subset_md=host_subset_md.set_index("SampleID")
host_subset_md

,sample_name_2,studyID,SampleID3,SampleID2,file_filtered_r1r2_combined,reads_per_unfiltered_file,reads_per_file,species_id,host_phylum,host_class,...,host_common_name,host_flight,host_modality,host_diet,country,habitat,sample_type,diet_category_12,diet_category_3,captive_wild
SampleID,,,,,,,,,,,,,,,,,,,,,
Bradypus_variegatus,13881.SAMEA6809531,13881.0,Bradypus-variegatus_SAMEA6809531.ebi_filtered_...,Bradypus-variegatus_SAMEA6809531.ebi_filtered_...,Bradypus-variegatus_SAMEA6809531.ebi_filtered_...,4687274.0,4681548.0,brown-throated_sloth.1,Chordata,Mammalia,...,Brown-throated sloth,NaN,NaN,herbivore,Nicaragua,NaN,feces,herbivore,herbivore,wild
Chalcites_basalis,13881.SAMEA6809488,13881.0,Chalcites-basalis_SAMEA6809488.ebi_filtered_r1...,Chalcites-basalis_SAMEA6809488.ebi_filtered_r1...,Chalcites-basalis_SAMEA6809488.ebi_filtered_r1...,2010544.0,2010106.0,horsfields_bronze_cuckoo.1,Chordata,Aves,...,Horsfields_Bronze_Cuckoo,NaN,NaN,carnivore,Australia,NaN,feces,insectivore,carnivore,wild
Choloepus_didactylus,13881.SAMEA6809544,13881.0,Choloepus-didactylus_SAMEA6809544.ebi_filtered...,Choloepus-didactylus_SAMEA6809544.ebi_filtered...,Choloepus-didactylus_SAMEA6809544.ebi_filtered...,11405774.0,11086930.0,southern_two-toed_sloth.1,Chordata,Mammalia,...,southern two-toed sloth,NaN,NaN,herbivore,Austria,NaN,feces,folivore,herbivore,captive
Choloepus_hoffmanni,13881.SAMEA6809527,13881.0,Choloepus-hoffmanni_SAMEA6809527.ebi_filtered_...,Choloepus-hoffmanni_SAMEA6809527.ebi_filtered_...,Choloepus-hoffmanni_SAMEA6809527.ebi_filtered_...,1983646.0,1983064.0,hoffmann's_two-fingered_sloth.1,Chordata,Mammalia,...,Hoffmann's two-fingered sloth,NaN,NaN,herbivore,Nicaragua,NaN,feces,herbivore,herbivore,wild
Eudromia_elegans,11166.NATL.25082.tinamou,11166.0,Eudromia-elegans_S_S_Temp_D706-AK1544_filtered...,Eudromia-elegans_S_S_Temp_D706-AK1544_filtered...,Eudromia-elegans_S_S_Temp_D706-AK1544_filtered...,52067270.0,51754556.0,tinamou.1,Chordata,Aves,...,tinamou,flighted,NaN,NaN,USA,NaN,NaN,omnivore,omnivore,captive
Gorilla_gorilla,13881.SAMEA6809515,13881.0,Gorilla-gorilla_SAMEA6809515.ebi_filtered_r1r2...,Gorilla-gorilla_SAMEA6809515.ebi_filtered_r1r2...,Gorilla-gorilla_SAMEA6809515.ebi_filtered_r1r2...,4459468.0,4411764.0,western_lowland_gorilla.2,Chordata,Mammalia,...,western lowland gorilla,NaN,NaN,herbivore,Republic of the Congo,NaN,feces,herbivore,herbivore,wild
Homo_sapiens,13881.SAMEA6809425,13881.0,Homo-sapiens_SAMEA6809425.ebi_filtered_r1r2int...,Homo-sapiens_SAMEA6809425.ebi_filtered_r1r2com...,Homo-sapiens_SAMEA6809425.ebi_filtered_r1r2com...,55200616.0,55172154.0,human.3,Chordata,Mammalia,...,human,NaN,NaN,omnivore,Austria,NaN,feces,omnivore,omnivore,human
Lemur_catta,13114.song.51.s004,13114.0,Lemur-catta_Song51_24592_host_filtered_r1r2int...,Lemur-catta_Song51_24592_host_filtered_r1r2com...,Lemur-catta_Song51_24592_host_filtered_r1r2com...,18513374.0,18510390.0,ringtailed_lemur.1,Chordata,Mammalia,...,Ringtailed lemur,flightless,"terrestrial, flightless",herbivore,USA,NaN,NaN,omnivore,omnivore,captive
Pteromys_volans,11166.AR.M18.flyingsquirrel,11166.0,Pteromys-volans_S_S_Temp_D707-AK1545_filtered_...,Pteromys-volans_S_S_Temp_D707-AK1545_filtered_...,Pteromys-volans_S_S_Temp_D707-AK1545_filtered_...,20695232.0,20495598.0,siberian_flying_squirrel.1,Chordata,Mammalia,...,Siberian flying squirrel,flightless,NaN,NaN,Finland,NaN,NaN,herbivore,herbivore,wild


In [115]:
host_metadata_fp = join(md_dir, "metadata_TimeTree_returned_as_SampleID.txt") ## Path to strainphlan tree
host_subset_md.to_csv(host_metadata_fp, sep='\t')

In [116]:
!qiime empress tree-plot \
    --i-tree $indir"/Akkermansia_muciniphila_subsampled_host_tree.qza"\
    --m-feature-metadata-file $host_metadata_fp \
    --o-visualization $outdir/A_muciniphila_host_tree.qzv

Saved Visualization to: /panfs/panfs1.ucsd.edu/panscratch/jhc103/VertMetaphlan-frmerged/empress-output/A_muciniphila_host_tree.qzv


In [117]:
## View the qza inline in jupyternotebook 
viz = Visualization.load('/panfs/panfs1.ucsd.edu/panscratch/jhc103/VertMetaphlan-frmerged/empress-output/A_muciniphila_host_tree.qzv')
viz

<visualization: Visualization uuid: 3a60694f-e56a-45af-9167-b9c48daa0de3>